# Halimbawa ng Sample Hotel at Flight Booker Agent

Ang solusyong ito ay makakatulong sa iyo na mag-book ng mga tiket sa eroplano at hotel. Ang senaryo ay isang biyahe mula London Heathrow LHR noong Pebrero 20, 2024 papuntang New York JFK, pabalik noong Pebrero 27, 2025, sakay ng economy class gamit lamang ang British Airways. Gusto kong manatili sa isang Hilton hotel sa New York, mangyaring ibigay ang mga gastos para sa flight at hotel.


# I-initialize ang Azure AI Agent Service at kunin ang impormasyon ng configuration mula sa **.env**

### **.env**

Gumawa ng .env file

Ang **.env** ay naglalaman ng connection string ng Azure AI Agent Service, ang modelong ginagamit ng AOAI, at ang kaukulang Google API Search service API, ENDPOINT, at iba pa.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Pangalan ng Deployment ng Modelo ng Iyong Azure AI Agent Service"

[**NOTE**] Kailangan mo ng modelo na may 100,000 Rate Limit (Tokens kada minuto) at Rate Limit na 600 (Request kada minuto).

  Maaari kang makakuha ng modelo sa Azure AI Foundry - Model at Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Connection String ng Iyong Azure AI Agent Service Project"

  Maaari mong makuha ang connection string ng proyekto sa iyong project overview sa AI Foundry Portal Screen.

- **SERPAPI_SEARCH_API_KEY** = "Ang Iyong SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Ang Iyong SERPAPI Search Endpoint"

Upang makuha ang Model Deployment Name at Project Connection String ng Azure AI Agent Service, kailangan mong gumawa ng Azure AI Agent Service. Inirerekomenda na gamitin ang [template na ito](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) upang direktang gawin ito. （***Tandaan:*** Ang Azure AI Agent Service ay kasalukuyang nakatakda sa limitadong rehiyon. Inirerekomenda na sumangguni ka sa [link na ito](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) upang itakda ang rehiyon.)

Kailangang ma-access ng Agent ang SERPAPI. Inirerekomenda na magparehistro gamit ang [link na ito](https://serpapi.com/searches). Pagkatapos ng pagpaparehistro, maaari kang makakuha ng natatanging API KEY at ENDPOINT.


# Setup

Upang patakbuhin ang notebook na ito, siguraduhin na na-install mo ang mga kinakailangang library sa pamamagitan ng pagtakbo ng `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Ang iyong Semantic Kernel na bersyon ay dapat hindi bababa sa 1.27.2.


I-load ang iyong .env file na mga setting at resources, pakitiyak na naidagdag mo na ang iyong mga key at setting at nakagawa ng lokal na .env file.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Mag-log in sa Azure

Kailangan mo nang mag-log in sa Azure. Buksan ang terminal at patakbuhin ang sumusunod na utos:

```bash
az login
```

Ang utos na ito ay hihilingin sa iyo na ilagay ang iyong Azure credentials, na magpapagana nang maayos sa Azure AI Agent service.


# Paliwanag:
Ito ay isang variable na nag-iimbak ng API key para sa pag-access sa isang SERP (Search Engine Results Page) API service. Ang API key ay isang natatanging identifier na ginagamit upang i-authenticate ang mga request na konektado sa iyong account.

Layunin: Ang layunin ng linyang ito ay iimbak ang API key sa isang variable upang magamit ito sa pag-authenticate ng mga request sa SERP API service. Kinakailangan ang API key upang ma-access ang serbisyo at makapagsagawa ng mga paghahanap.

Paano Kumuha ng SERP API Key: Upang makakuha ng SERP API key, sundin ang mga pangkalahatang hakbang sa https://serpapi.com (ang eksaktong mga hakbang ay maaaring magkaiba depende sa partikular na SERP API service na ginagamit mo):

Pumili ng SERP API Service: Mayroong iba't ibang SERP API services na magagamit, tulad ng SerpAPI, Google Custom Search JSON API, at iba pa. Piliin ang pinakaangkop sa iyong pangangailangan.

Magrehistro para sa isang Account: Pumunta sa website ng napiling SERP API service at magrehistro para sa isang account. Maaaring kailanganin mong magbigay ng ilang pangunahing impormasyon at i-verify ang iyong email address.

Gumawa ng API Key: Pagkatapos magrehistro, mag-log in sa iyong account at pumunta sa API section o dashboard. Hanapin ang opsyon upang lumikha o mag-generate ng bagong API key.  
Kopyahin ang API Key sa iyong .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Paliwanag:
BASE_URL: Ito ay isang variable na nag-iimbak ng base URL para sa SERP API endpoint. Ang pangalan ng variable na BASE_URL ay isang convention na ginagamit upang ipakita na ang URL na ito ang panimulang punto para sa paggawa ng mga API request.  
'https://serpapi.com/search':  

Ito ang aktwal na URL string na itinalaga sa BASE_URL variable. Kinakatawan nito ang endpoint para sa pagsasagawa ng mga search query gamit ang SERP API.

# Layunin:
Ang layunin ng linyang ito ay magtakda ng isang constant na naglalaman ng base URL para sa SERP API. Ang URL na ito ang gagamitin bilang panimulang punto para sa pagbuo ng mga API request upang magsagawa ng mga search operation.

# Paggamit:
Sa pamamagitan ng pagtukoy sa base URL sa isang variable, madali mo itong magagamit muli sa iyong code tuwing kailangan mong gumawa ng mga request sa SERP API. Ginagawa nitong mas madaling i-maintain ang iyong code at binabawasan ang panganib ng mga error mula sa pag-hardcode ng URL sa maraming lugar. Ang kasalukuyang halimbawa ay https://serpapi.com/search?engine=bing na gumagamit ng Bing search API. Maaaring pumili ng ibang API sa https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Paliwanag:

Dito matatagpuan ang iyong plugin code.

Class Definition: `class BookingPlugin`: Nagde-define ng isang klase na tinatawag na BookingPlugin na naglalaman ng mga method para sa pag-book ng mga hotel at flight.

Paraan ng Pag-book ng Hotel:

- `@kernel_function(description="booking hotel")`: Isang decorator na naglalarawan sa function bilang kernel function para sa pag-book ng mga hotel.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Nagde-define ng isang method para sa pag-book ng mga hotel na may mga annotated na parameter at return type.

Ang method ay bumubuo ng isang dictionary ng mga parameter para sa hotel booking request at nagpapadala ng GET request sa SERP API. Sinusuri nito ang response status at ibinabalik ang mga property ng hotel kung matagumpay, o None kung nabigo ang request.

Paraan ng Pag-book ng Flight:

- `@kernel_function(description="booking flight")`: Isang decorator na naglalarawan sa function bilang kernel function para sa pag-book ng mga flight.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Nagde-define ng isang method para sa pag-book ng mga flight na may mga annotated na parameter at return type.

Ang method ay bumubuo ng mga dictionary ng mga parameter para sa outbound at return flight requests at nagpapadala ng GET requests sa SERP API. Sinusuri nito ang response status at idinadagdag ang impormasyon ng flight sa result string kung matagumpay, o nagpi-print ng error message kung nabigo ang request. Ang method ay nagbabalik ng result string na naglalaman ng impormasyon ng flight.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Paliwanag:
Mga Import Statement: Mag-import ng mga kinakailangang module para sa Azure credentials, AI agent, nilalaman ng chat message, role ng author, at kernel function decorator.

Asynchronous Context Manager: `async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: Nagsesetup ito ng asynchronous context manager upang hawakan ang Azure credentials at lumikha ng AI agent client.

Pangalan ng Ahente at Mga Instruksyon:
- `AGENT_NAME = "BookingAgent"`: Tinutukoy ang pangalan ng ahente.
- `AGENT_INSTRUCTIONS = """..."""`: Nagbibigay ng detalyadong mga tagubilin para sa ahente kung paano hahawakan ang mga kahilingan sa booking.

Paglikha ng Agent Definition: `agent_definition = await client.agents.create_agent(...)`: Lumilikha ng agent definition gamit ang tinukoy na modelo, pangalan, at mga tagubilin.

Paglikha ng AzureAI Agent: `agent = AzureAIAgent(...)`: Lumilikha ng AzureAI agent gamit ang client, agent definition, at ang tinukoy na plugin.

Paglikha ng Thread: `thread: AzureAIAgentThread | None = None`: Lumilikha ng thread para sa ahente. Hindi kinakailangang gumawa muna ng thread - kung ang halaga ay `None`, isang bagong thread ang awtomatikong malilikha sa unang invocation at ibabalik bilang bahagi ng tugon.

Mga Input ng User: `user_inputs = ["..."]`: Tinutukoy ang listahan ng mga input ng user na ipoproseso ng ahente.

Sa `finally` block, tanggalin ang thread at ahente upang linisin ang mga resources.


# Pagpapatotoo

Ang `DefaultAzureCredential` na klase ay bahagi ng Azure SDK para sa Python. Nagbibigay ito ng default na paraan upang magpatotoo sa mga serbisyo ng Azure. Sinusubukan nitong magpatotoo gamit ang iba't ibang mga pamamaraan sa isang tiyak na pagkakasunod-sunod, tulad ng mga environment variable, managed identity, at Azure CLI credentials.

Mga Asynchronous na Operasyon: Ang aio module ay nagpapahiwatig na ang `DefaultAzureCredential` na klase ay sumusuporta sa mga asynchronous na operasyon. Ibig sabihin, maaari mo itong gamitin kasama ng asyncio upang magsagawa ng mga non-blocking na kahilingan sa pagpapatotoo.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
